## Criação de dicionários com notas dos usuários e filmes para desenvolvimento do sistema de recomendação do zero, sem utilização de API já estabelecida.



In [6]:
avaliacoesUsuario = {'Ana': 
        {'Freddy x Jason': 2.5, 
        'O Ultimato Bourne': 3.5,
         'Star Trek': 3.0, 
        'Exterminador do Futuro': 3.5, 
         'Norbit': 2.5, 
         'Star Wars': 3.0},
        'Marcos': 
        {'Freddy x Jason': 3.0, 
         'O Ultimato Bourne': 3.5, 
        'Star Trek': 1.5, 
         'Exterminador do Futuro': 5.0, 
        'Star Wars': 3.0, 
         'Norbit': 3.5}, 

        'Pedro': 
        {'Freddy x Jason': 2.5, 
         'O Ultimato Bourne': 3.0,
        'Exterminador do Futuro': 3.5, 
         'Star Wars': 4.0},
                     
        'Claudia': 
        {'O Ultimato Bourne': 3.5, 
         'Star Trek': 3.0,
        'Star Wars': 4.5, 
         'Exterminador do Futuro': 4.0, 
        'Norbit': 2.5},
        'Adriano': 
        {'Freddy x Jason': 3.0, 
         'O Ultimato Bourne': 4.0, 
        'Star Trek': 2.0, 
         'Exterminador do Futuro': 3.0, 
        'Star Wars': 3.0,
         'Norbit': 2.0}, 
        'Janaina': 
        {'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 4.0,
        'Star Wars': 3.0, 
        'Exterminador do Futuro': 5.0, 
        'Norbit': 3.5},  
        'Leonardo': 
        {'O Ultimato Bourne':4.5,
        'Exterminador do Futuro':4.0}
}

Dicionário de avaliações por item, contemplando os Scores dos usuários

In [7]:
avaliacoesFilme = {'Freddy x Jason': 
        {'Ana': 2.5, 
         'Marcos:': 3.0 ,
        'Pedro': 2.5, 
        'Adriano': 3.0, 
         'Janaina': 3.0 },

        'O Ultimato Bourne': 
        {'Ana': 3.5, 
         'Marcos': 3.5,
        'Pedro': 3.0, 
         'Claudia': 3.5, 
        'Adriano': 4.0, 
         'Janaina': 4.0,
        'Leonardo': 4.5 },
        'Star Trek': 
        {'Ana': 3.0, 
         'Marcos:': 1.5,
        'Claudia': 3.0, 
         'Adriano': 2.0 },
    
        'Exterminador do Futuro': 
        {'Ana': 3.5, 
         'Marcos:': 5.0 ,
        'Pedro': 3.5, 
         'Claudia': 4.0, 
        'Adriano': 3.0, 
         'Janaina': 5.0,
        'Leonardo': 4.0},
 
        'Norbit': 
        {'Ana': 2.5, 
        'Marcos:': 3.0 ,
        'Claudia': 2.5, 
        'Adriano': 2.0, 
        'Janaina': 3.5,
        'Leonardo': 1.0},
         
        'Star Wars': 
        {'Ana': 3.0, 
        'Marcos:': 3.5,
        'Pedro': 4.0, 
        'Claudia': 4.5, 
        'Adriano': 3.0, 
        'Janaina': 3.0}
}

Importação da Biblioteca math para utilização da função sqrt

In [43]:
from math import sqrt

Em matemática, distância euclidiana (ou distância métrica) é a distância entre dois pontos, 
que pode ser provada pela aplicação repetida do teorema de Pitágoras

In [44]:
def euclidiana(base, usuario1, usuario2):
    si = {}
    for item in base[usuario1]:
       if item in base[usuario2]: si[item] = 1

    if len(si) == 0: return 0

    soma = sum([pow(base[usuario1][item] - base[usuario2][item], 2)
                for item in base[usuario1] if item in base[usuario2]])
    return 1/(1 + sqrt(soma))


Função para agrupar as similaridades dos usuários, contemplando IFs para impedir que o mesmo usuário seja avaliado novamente

In [29]:
def getSimilares(base, usuario):
    similaridade = [(euclidiana(base, usuario, outro), outro)
                    for outro in base if outro != usuario]
    similaridade.sort()
    similaridade.reverse()
    return similaridade[0:30]

In [45]:
def getRecomendacoesUsuario(base, usuario):
    totais={}
    somaSimilaridade={}
    for outro in base:
        if outro == usuario: continue
        similaridade = euclidiana(base, usuario, outro)

        if similaridade <= 0: continue

        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade
    rankings=[(total / somaSimilaridade[item], item) for item, total in totais.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[0:30]

Utilizado o conjunto de dados MovieLens do Projeto de Pesquisa GroupLens na Universidade de Minnesota.
 Este conjunto de dados consiste em:
* 100.000 avaliações (1-5) de 943 usuários em 1682 filmes.
* Cada usuário classificou pelo menos 20 filmes.
Os dados foram coletados através do site da MovieLens
(movielens.umn.edu) durante o período de sete meses a partir de 19 de setembro,
1997 até 22 de abril de 1998. Esses dados foram limpos - usuários
que tinha menos de 20 classificações ou não tinha demografia completa
informações foram removidas deste conjunto de dados. Descrições detalhadas de
o arquivo de dados pode ser encontrado no final deste arquivo.

In [31]:
def carregaMovieLens(path='C:/Users/gs70/UDEMY/Recomendacao Jones'):
    filmes = {}
    for linha in open(path + '/u.item'):
        (id, titulo) = linha.split('|')[0:2]
        filmes[id] = titulo
    # print(filmes)

    base = {}
    for linha in open(path + '/u.data'):
        (usuario, idfilme, nota, tempo) = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][filmes[idfilme]] = float(nota)
    return base            

In [32]:
def calculaItensSimilares(base):
    result = {}
    for item in base:
        notas = getSimilares(base, item)
        result[item] = notas
    return result

In [46]:
def getRecomendacoesItens(baseUsuario, similaridadeItens, usuario):
    notasUsuario = baseUsuario[usuario]
    notas={}
    totalSimilaridade={}
    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridadeItens[item]:
            if item2 in notasUsuario: continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2,0)
            totalSimilaridade[item2] += similaridade
    rankings=[(score/totalSimilaridade[item], item) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()
    return rankings

In [35]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Leonardo')

[(3.418471702396272, 'Star Wars'),
 (2.8062345567968117, 'Freddy x Jason'),
 (2.7790163740175453, 'Norbit'),
 (2.4116411912741587, 'Star Trek')]

In [41]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Pedro')

[(2.7271986443555543, 'Norbit'), (2.4825817086405517, 'Star Trek')]

Lista de dados que estaria pré-armazenada (calculada) para disparar recomendações com tempo satisfatório aos usuários

In [40]:
listaitens = calculaItensSimilares(avaliacoesFilme)

In [42]:
getRecomendacoesItens(avaliacoesUsuario, listaitens,'Leonardo')

[(4.3031757536216695, 'Star Trek'),
 (4.2987972584088565, 'Freddy x Jason'),
 (4.271774713337376, 'Star Wars'),
 (4.25267605996656, 'Norbit')]

Verificando as recomendações por usuários e itens.
Definido threshold de 30 filmes do ranking.
Escolhido aleatóriamente o usuário 200 para listar os usuário similares

In [48]:
base = carregaMovieLens()

In [49]:
getSimilares(base, '200')

[(1.0, '688'),
 (0.5, '856'),
 (0.5, '775'),
 (0.5, '762'),
 (0.5, '558'),
 (0.5, '384'),
 (0.4142135623730951, '845'),
 (0.4142135623730951, '818'),
 (0.4142135623730951, '809'),
 (0.4142135623730951, '36'),
 (0.4142135623730951, '355'),
 (0.4142135623730951, '179'),
 (0.4142135623730951, '147'),
 (0.4142135623730951, '111'),
 (0.36602540378443865, '909'),
 (0.36602540378443865, '9'),
 (0.36602540378443865, '858'),
 (0.36602540378443865, '855'),
 (0.36602540378443865, '740'),
 (0.36602540378443865, '701'),
 (0.36602540378443865, '675'),
 (0.36602540378443865, '616'),
 (0.36602540378443865, '520'),
 (0.36602540378443865, '390'),
 (0.36602540378443865, '33'),
 (0.36602540378443865, '241'),
 (0.36602540378443865, '131'),
 (0.36602540378443865, '105'),
 (0.3333333333333333, '888'),
 (0.3333333333333333, '784')]